In [1]:
import logging
import socket
from confluent_kafka import Producer, Consumer
import json
import os
from joblib import load
import logging
import numpy as np

In [2]:
NUM_PARTITIONS = 3
KAFKA_BROKER = "kafka:9092"
TRANSACTIONS_TOPIC = "transactions"
TRANSACTIONS_CONSUMER_GROUP = "transactions"
ANOMALIES_TOPIC = "anomalies"

In [ ]:
def create_producer():
    try:
        producer = Producer({"bootstrap.servers": KAFKA_BROKER,
                             "client.id": socket.gethostname(),
                             "enable.idempotence": True,  # EOS processing
                             "compression.type": "lz4",
                             "batch.size": 64000,
                             "linger.ms": 10,
                             "acks": "all",  # Wait for the leader and all ISR to send response back
                             "retries": 5,
                             "delivery.timeout.ms": 1000})  # Total time to make retries
    except Exception as e:
        logging.exception("Couldn't create the producer")
        producer = None
    return producer


def create_consumer(topic, group_id):
    try:
        consumer = Consumer({"bootstrap.servers": KAFKA_BROKER,
                             "group.id": group_id,
                             "client.id": socket.gethostname(),
                             "isolation.level": "read_committed",
                             "default.topic.config": {"auto.offset.reset": "latest", # Only consume new messages
                                                      "enable.auto.commit": False}
                             })

        consumer.subscribe([topic])
    except Exception as e:
        logging.exception("Couldn't create the consumer")
        consumer = None

    return consumer

model_path = os.path.abspath('./isolation_forest.joblib')


def detect():
    consumer = create_consumer(topic=TRANSACTIONS_TOPIC, group_id=TRANSACTIONS_CONSUMER_GROUP)

    producer = create_producer()

    clf = load(model_path)

    while True:
        message = consumer.poll(timeout=50)
        if message is None:
            continue
        if message.error():
            logging.error("Consumer error: {}".format(message.error()))
            continue

        # Message that came from producer
        record = json.loads(message.value().decode('utf-8'))
        data = record["data"]
        print(data)
        prediction = clf.predict(data)
        if prediction[0] == 1:
            print('Normal')

        # prediction = clf.predict(data)

        # If an anomaly comes in, send it to anomalies topic
        if prediction[0] == -1:
            print('Abnormal')
            score = clf.score_samples(data)
            record["score"] = np.round(score, 3).tolist()

            _id = str(record["id"])
            record = json.dumps(record).encode("utf-8")

            producer.produce(topic=ANOMALIES_TOPIC,
                                value=record)
            producer.flush()
            print(record)
            print('Alert send')
try:
    for _ in range(NUM_PARTITIONS):
        detect()
except KeyboardInterrupt:
    print("Break")

[[-2.013, -2.021]]
Normal
[[2.019, 1.741]]
Normal
[[-1.913, 2.532]]
Abnormal
b'{"id": 12, "data": [[-1.913, 2.532]], "current_time": "2025-10-27T09:05:05.689409", "score": [-0.708]}'
Alert send
[[0.1, -0.168]]
Abnormal
b'{"id": 13, "data": [[0.1, -0.168]], "current_time": "2025-10-27T09:05:07.722016", "score": [-0.739]}'
Alert send
[[1.982, 2.029]]
Normal
[[0.384, -2.374]]
Abnormal
b'{"id": 15, "data": [[0.384, -2.374]], "current_time": "2025-10-27T09:05:11.870614", "score": [-0.675]}'
Alert send
[[-1.992, -1.865]]
Normal
[[1.874, 2.221]]
Normal
[[-1.753, -2.34]]
Normal
[[1.483, 1.3]]
Normal
[[-1.734, -2.158]]
Normal
[[1.742, 1.984]]
Normal
[[0.9, -1.499]]
Abnormal
b'{"id": 22, "data": [[0.9, -1.499]], "current_time": "2025-10-27T09:05:26.230357", "score": [-0.709]}'
Alert send
[[0.711, -3.137]]
Abnormal
b'{"id": 23, "data": [[0.711, -3.137]], "current_time": "2025-10-27T09:05:28.245095", "score": [-0.725]}'
Alert send
[[-2.52, -2.325]]
Normal
[[-1.964, -2.37]]
Normal
[[-1.989, -1.843]